# Description

Like `07_01` but using more CPU cores.

# Use multiple CPU core

In [1]:
%env CM_N_JOBS=2
%env NUMBA_NUM_THREADS=2
%env MKL_NUM_THREADS=2
%env OPEN_BLAS_NUM_THREADS=2
%env NUMEXPR_NUM_THREADS=2
%env OMP_NUM_THREADS=2

env: CM_N_JOBS=2
env: NUMBA_NUM_THREADS=2
env: MKL_NUM_THREADS=2
env: OPEN_BLAS_NUM_THREADS=2
env: NUMEXPR_NUM_THREADS=2
env: OMP_NUM_THREADS=2


# Disable numba

In [2]:
%env NUMBA_DISABLE_JIT=1

env: NUMBA_DISABLE_JIT=1


# Remove pycache dir

In [3]:
!echo ${CODE_DIR}

/opt/code


In [4]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

/opt/code/libs/ccc/coef/__pycache__
/opt/code/libs/ccc/pytorch/__pycache__
/opt/code/libs/ccc/scipy/__pycache__
/opt/code/libs/ccc/utils/__pycache__
/opt/code/libs/ccc/__pycache__
/opt/code/libs/ccc/sklearn/__pycache__


In [5]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -prune -exec rm -rf {} \;

In [6]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

# Modules

In [7]:
import numpy as np

from ccc.coef import ccc

In [8]:
# let numba compile all the code before profiling
ccc(np.random.rand(10), np.random.rand(10))

0.20454545454545456

# Data

In [9]:
n_genes, n_samples = 10, 30000

In [10]:
np.random.seed(0)

In [11]:
data = np.random.rand(n_genes, n_samples)

In [12]:
data.shape

(10, 30000)

# With default `internal_n_clusters`

In [13]:
def func():
    n_clust = list(range(2, 10 + 1))
    return ccc(data, internal_n_clusters=n_clust, n_jobs=2)

In [14]:
%%timeit func()
func()

2.39 s ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


These results are just slightly worse than the numba-compiled version (notebook `07`).

# With reduced `internal_n_clusters`

In [15]:
def func():
    n_clust = list(range(2, 5 + 1))
    return ccc(data, internal_n_clusters=n_clust, n_jobs=2)

In [16]:
%%timeit func()
func()

303 ms ± 3.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


These results are slightly better than the numba-compiled version (notebook `07`), which is surprising. In the future, it would be interesting to disable threading here to get accurate profiling results to debug this issue.